# chargement des fichiers csv en noms de variables

In [178]:
import pandas as pd


meteo= pd.read_csv("../../../csv_modélisé/all_meteo.csv")
objets= pd.read_csv("../../../csv_modélisé/objets_trouves.csv")
frequentation= pd.read_csv("../../../csv_modélisé/frequentation.csv")

# Enlever éventuels doublons dans les données

In [179]:
meteo=meteo.drop_duplicates()
frequentation=frequentation.drop_duplicates()
objets=objets.drop_duplicates()

# Filtrage des champs d'intérêt

In [180]:
meteo=meteo.loc[:, ['DATE',  'CLOUDCOVER_AVG_PERCENT', 'SUNHOUR']]

In [181]:
objets.head()

,date,type,gare
0,2021-01-01,NaN,Paris Gare de Lyon
1,2021-01-02,NaN,Paris Gare de Lyon
4,2021-01-03,NaN,Paris Gare de Lyon
6,2021-01-04,NaN,Paris Gare de Lyon
8,2021-01-05,NaN,Paris Gare de Lyon


# Arrondir les valeures des données météo

In [182]:
import pandas as pd

# Exemple de DataFrame original
# meteo = pd.DataFrame(...)

# Arrondir les valeurs des colonnes spécifiques
columns_to_round = [ 'CLOUDCOVER_AVG_PERCENT', 'SUNHOUR']
meteo[columns_to_round] = meteo[columns_to_round].round().astype(int)

# Crée un nouveau DataFrame avec les colonnes spécifiques et les données arrondies
new_meteo = meteo[['DATE'] + columns_to_round]

# Afficher les premières lignes du nouveau DataFrame pour vérification
meteo = new_meteo[['DATE', 'CLOUDCOVER_AVG_PERCENT', 'SUNHOUR']]
print(meteo.head(2000))

            DATE  CLOUDCOVER_AVG_PERCENT  SUNHOUR
0     2021-01-01                      83        3
1     2021-01-02                      55        9
2     2021-01-03                      71        9
3     2021-01-04                     100        3
4     2021-01-05                      98        3
...          ...                     ...      ...
1090  2023-12-27                      42        7
1091  2023-12-28                      61        5
1092  2023-12-29                      86        1
1093  2023-12-30                      55        5
1094  2023-12-31                      81        3

[1095 rows x 3 columns]


# garder 10 premiers caractéres des lignes date

In [183]:
objets['date'] = objets['date'].str[:10]
# va permettre d'avoir un format de date commun à la table météo
objets.head(5)


,date,type,gare
0,2021-01-01,NaN,Paris Gare de Lyon
1,2021-01-02,NaN,Paris Gare de Lyon
4,2021-01-03,NaN,Paris Gare de Lyon
6,2021-01-04,NaN,Paris Gare de Lyon
8,2021-01-05,NaN,Paris Gare de Lyon


# mappage des valeures du champ type pour faciliter analyse

In [184]:
mappage = {
    'Bagagerie: sacs, valises, cartables': 'bagages',
    'Porte-monnaie / portefeuille, argent, titres': 'clé',
    'Appareils électroniques, informatiques, appareils photo': 'electronique',
    'Articles médicaux': 'médical',
    'Pièces d identités et papiers personnels': 'papiers',
    'Clés, porte-clés, badge magnétique': 'clé',
    'Optique': 'optique',
    'Divers': 'divers',
    'Articles d enfants, de puériculture': 'articles enfants',
    'Vêtements, chaussures': 'vetements',
    'Vélos, trottinettes, accessoires 2 roues': 'velo/trottinette',
    'Instruments de musique': 'article musique',
    'Bijoux, montres': 'bijoux/montres',
    'Articles de sport, loisirs, camping': 'sport/loisirs',
    'Livres, articles de papéterie': 'livres',
    'Parapluies': 'parapluies'
}

# Appliquer le mappage à la colonne 'type'
objets['type'] = objets['type'].map(mappage)

# Afficher les premières lignes du DataFrame mis à jour pour vérification
print(objets.head())



         date type                gare
0  2021-01-01  NaN  Paris Gare de Lyon
1  2021-01-02  NaN  Paris Gare de Lyon
4  2021-01-03  NaN  Paris Gare de Lyon
6  2021-01-04  NaN  Paris Gare de Lyon
8  2021-01-05  NaN  Paris Gare de Lyon


# actualisation des fichiers csv

In [185]:
valeurs_uniques = objets['type'].unique()

# Afficher les valeurs uniques
print("Valeurs uniques dans la colonne 'type' :")
print(valeurs_uniques)

Valeurs uniques dans la colonne 'type' :
[nan]


In [186]:
meteo.to_csv('../../../csv_modélisé/all_meteo.csv', index=False)
objets.to_csv('../../../csv_modélisé/objets_trouves_bis.csv', index=False)
frequentation.to_csv('../../../csv_modélisé/frequentation.csv', index=False)

In [187]:
meteo.columns

Index(['DATE', 'CLOUDCOVER_AVG_PERCENT', 'SUNHOUR'], dtype='object')

In [188]:
import sqlite3

# Connexion à la base de données
conn = sqlite3.connect('bdd_luminosite.db')

# Créer un curseur
curseur = conn.cursor()


    # Insérer les données dans la table 'lumiere'
for index, row in meteo.iterrows():
    curseur.execute("INSERT INTO lumiere (date, cloud, sun) VALUES (?, ?, ?)", 
                    (row['DATE'], row['CLOUDCOVER_AVG_PERCENT'], row['SUNHOUR']))

# Insérer les données dans la table 'frequentation'
for index, row in frequentation.iterrows():
    curseur.execute("INSERT INTO frequentation (nom_gare, frequent_2021, frequent_2022, frequent_2023) VALUES (?, ?, ?, ?)", 
                    (row['gare'], row['frequent_2021'], row['frequent_2022'], row['frequent_2023']))

# Insérer les données dans la table 'objets_trouves'
for index, row in objets.iterrows():
    curseur.execute("INSERT INTO objets_trouves (date, type, nom_gare) VALUES (?, ?, ?)", 
                    (row['date'], row['type'], row['gare']))

# Valider toutes les transactions en une fois
conn.commit() 


    # Fermer la connexion
conn.close()

print("Données insérées avec succès dans la base de données.")


IntegrityError: UNIQUE constraint failed: lumiere.date

# normalisation des données 

In [ ]:
objets.head(30)

,date,type,gare
0,2021-01-01,NaN,Paris Gare de Lyon
1,2021-01-02,NaN,Paris Gare de Lyon
3,2021-01-02,NaN,Paris Gare de Lyon
6,2021-01-02,NaN,Paris Gare de Lyon
15,2021-01-03,NaN,Paris Gare de Lyon
16,2021-01-03,NaN,Paris Gare de Lyon
19,2021-01-04,NaN,Paris Gare de Lyon
20,2021-01-04,NaN,Paris Gare de Lyon
21,2021-01-05,NaN,Paris Gare de Lyon
22,2021-01-05,NaN,Paris Gare de Lyon


In [ ]:
frequentation.head(10)   

,Unnamed: 0,gare,frequent_2021,frequent_2022,frequent_2023
0,0,Paris Gare de Lyon,189125448,231360748,251046517
1,1,Paris Montparnasse,92442771,128082169,145228277
2,2,Paris Gare du Nord,275838875,451647681,482925909
3,3,Paris Est,62800592,83197515,90216815
4,4,Paris Saint-Lazare,213505115,240124708,241391780
5,5,Paris Austerlitz,45096165,39824989,44259883
6,6,Paris Bercy Bourgogne - Pays d'Auvergne,6821651,9710201,10340177


In [ ]:
import pandas as pd
import sqlite3
import os

# Obtenir le répertoire du script et construire le chemin pour la base de données
repertoire_script = os.getcwd()  # Obtenir le répertoire de travail courant
chemin_bdd = os.path.join(repertoire_script, 'bdd_luminosite.db')

# Créer une connexion à la base de données
conn = sqlite3.connect(chemin_bdd)

# Charger les données de la table 'frequentation' dans un DataFrame
df_frequentation = pd.read_sql_query("SELECT * FROM frequentation", conn)

# Charger les données de la table 'objets_trouves' dans un DataFrame
df_objets = pd.read_sql_query("SELECT * FROM objets_trouves", conn)

# Fermer la connexion après la lecture
conn.close()

# Normaliser les données de fréquentation
# Référence de fréquentation la plus faible pour 2021
frequentation_min = df_frequentation['frequent_2021'].min()

# Ajouter un champ de comptage normalisé pour chaque année
df_frequentation['frequent_2021_norm'] = df_frequentation['frequent_2021'] / frequentation_min
df_frequentation['frequent_2022_norm'] = df_frequentation['frequent_2022'] / frequentation_min
df_frequentation['frequent_2023_norm'] = df_frequentation['frequent_2023'] / frequentation_min

# Convertir les dates en années pour correspondre avec les données de fréquentation
df_objets['date'] = pd.to_datetime(df_objets['date'])
df_objets['année'] = df_objets['date'].dt.year

# Fusionner les données de fréquentation avec les données d'objets
df_merged = pd.merge(df_objets, df_frequentation, left_on='nom_gare', right_on='nom_gare')

# Ajouter la colonne de comptage normalisé en fonction de l'année
df_merged['comptage_normalisé'] = df_merged.apply(
    lambda row: row[f'frequent_{row["année"]}_norm'], axis=1
)

# Calculer la somme des comptages normalisés pour chaque date
df_normalized = df_merged.groupby('date').agg({'comptage_normalisé': 'sum'}).reset_index()
df_normalized.rename(columns={'comptage_normalisé': 'comptage_normalisé_total'}, inplace=True)

# Fusionner avec le DataFrame des objets
df_final = pd.merge(df_merged, df_normalized, on='date')

# Exporter les résultats dans un fichier CSV
df_final.to_csv('objets_trouves_normalises.csv', index=False)

print("Les données ont été normalisées et sauvegardées avec succès.")



Les données ont été normalisées et sauvegardées avec succès.


In [ ]:
import pandas as pd
import sqlite3
import os

# Obtenir le répertoire du script et construire le chemin pour la base de données
repertoire_script = os.getcwd()  # Obtenir le répertoire de travail courant
chemin_bdd = os.path.join(repertoire_script, 'bdd_luminosite.db')

# Créer une connexion à la base de données
conn = sqlite3.connect(chemin_bdd)

# Charger les données de la table 'frequentation' dans un DataFrame
df_frequentation = pd.read_sql_query("SELECT * FROM frequentation", conn)

# Charger les données de la table 'objets_trouves' dans un DataFrame
df_objets = pd.read_sql_query("SELECT * FROM objets_trouves", conn)

# Fermer la connexion après la lecture
conn.close()

# Convertir les dates en années pour correspondre avec les données de fréquentation
df_objets['date'] = pd.to_datetime(df_objets['date'])
df_objets['année'] = df_objets['date'].dt.year

# Fusionner les données de fréquentation avec les données d'objets
df_merged = pd.merge(df_objets, df_frequentation, left_on='nom_gare', right_on='nom_gare')

# Ajouter une colonne de comptage normalisé en fonction de l'année
df_merged['frequent_norm'] = df_merged.apply(
    lambda row:  df_frequentation['frequent_2021'].min()/row[f'frequent_{row["année"]}'], axis=1
)

# Calculer la pondération pour chaque objet
df_merged['comptage_normalisé'] = df_merged['frequent_norm'].round(2)

# Réorganiser les colonnes
df_final = df_merged[['date', 'type', 'nom_gare', 'comptage_normalisé']]

# Exporter les résultats dans un fichier CSV
df_final.to_csv('objets_trouves_ponderes.csv', index=False)

print("Les données ont été pondérées et sauvegardées avec succès.")


Les données ont été pondérées et sauvegardées avec succès.


In [ ]:
frequentation.head()  

,Unnamed: 0,gare,frequent_2021,frequent_2022,frequent_2023
0,0,Paris Gare de Lyon,189125448,231360748,251046517
1,1,Paris Montparnasse,92442771,128082169,145228277
2,2,Paris Gare du Nord,275838875,451647681,482925909
3,3,Paris Est,62800592,83197515,90216815
4,4,Paris Saint-Lazare,213505115,240124708,241391780


In [ ]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('bdd_luminosite.db')

# Load data from the database
objets = pd.read_sql_query("SELECT * FROM objets_trouves", conn)
frequentation = pd.read_sql_query("SELECT * FROM frequentation", conn)

# Close the connection
conn.close()

# Convert 'date' column in objets to datetime format
objets['date'] = pd.to_datetime(objets['date'])

# Add year column to objets
objets['year'] = objets['date'].dt.year

# Define a function to calculate normalized count
def calculate_normalized_count(row, frequentation_df):
    year = row['year']
    gare = row['nom_gare']
    
    # Get the frequentation for the gare and year
    gare_frequentation = frequentation_df[
        (frequentation_df['nom_gare'] == gare)
    ]
    
    if gare_frequentation.empty:
        return None
    
    # Select the column for the correct year
    frequentation_column = f'frequent_{year}'
    
    # Ensure the column exists in the DataFrame
    if frequentation_column not in gare_frequentation.columns:
        return None
    
    gare_frequentation_value = gare_frequentation.iloc[0][frequentation_column]
    
    # Get the minimum frequentation value for normalization
    min_frequentation_value = frequentation_df[[col for col in frequentation_df.columns if col.startswith('frequent_')]].min().min()  # Minimum of all frequentations
    
    return gare_frequentation_value / min_frequentation_value

# Apply the function to calculate the normalized count
objets['comptage_normalise'] = objets.apply(lambda row: calculate_normalized_count(row, frequentation), axis=1)

# Save the updated DataFrame to a CSV file
csv_filename = 'objets_trouves_normalises11.csv'
objets.to_csv(csv_filename, index=False)

print(f"Data saved to {csv_filename}")



Data saved to objets_trouves_normalises11.csv


In [ ]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('bdd_luminosite.db')
curseur = conn.cursor()

# Function to count rows in a table
def count_rows(table_name):
    curseur.execute(f"SELECT COUNT(*) FROM {table_name}")
    count = curseur.fetchone()[0]
    return count

# List of tables to check
tables = ['frequentation', 'lumiere', 'objets_trouves']

# Count rows in each table
for table in tables:
    row_count = count_rows(table)
    print(f"Table {table} has {row_count} rows.")

# Close the connection
conn.close()

Table frequentation has 7 rows.
Table lumiere has 1095 rows.
Table objets_trouves has 1864 rows.


In [ ]:
obj_new=pd.read_csv("objets_trouves_ponderes.csv")
obj_new.head()

,date,type,nom_gare,comptage_normalisé
0,2021-01-01,NaN,Paris Gare de Lyon,0.04
1,2021-01-02,NaN,Paris Gare de Lyon,0.04
2,2021-01-02,NaN,Paris Gare de Lyon,0.04
3,2021-01-02,NaN,Paris Gare de Lyon,0.04
4,2021-01-03,NaN,Paris Gare de Lyon,0.04
